In [ ]:
# ====== RUN THIS CELL IN COLAB (complete debug + safe-fallback pipeline) ======
!pip install --quiet gspread oauth2client moviepy gTTS requests pillow

import os, time, textwrap, shutil, requests, random, glob
from datetime import datetime
from google.colab import files, drive
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from moviepy.editor import VideoFileClip, AudioFileClip, ImageClip, CompositeVideoClip, ColorClip
from gtts import gTTS
from PIL import Image, ImageDraw, ImageFont

# -------- CONFIG ----------
PIXABAY_API_KEY = "52223959-ad6e5aba93e33826921c7fedd"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1RZU0zDru5xZ8pbac7BhZnIdNBoRDprgwt9XsXEhEi4s/edit"
TAB_NAME = "VideoTopics"
DEBUG_TAB = "DEBUG_AUTOTEST"
LOCAL_DIR = "/content/videos"
DRIVE_FOLDER = "/content/drive/MyDrive/PixabayVideos"
os.makedirs(LOCAL_DIR, exist_ok=True)

# -------- Mount Drive --------
print("Mounting Drive...")
drive.mount('/content/drive', force_remount=True)
os.makedirs(DRIVE_FOLDER, exist_ok=True)
print("Drive folder ready:", DRIVE_FOLDER)

# -------- Upload service account JSON & auth sheets --------
print("\n📂 Upload service account JSON now")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("You must upload the service account JSON file.")
creds_filename = list(uploaded.keys())[0]
print("Using:", creds_filename)

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_filename, scope)
gc = gspread.authorize(creds)
print("gspread authorized")

# -------- Open spreadsheet and worksheet --------
try:
    sh = gc.open_by_url(SPREADSHEET_URL)
    print("Spreadsheet title:", sh.title)
    names = [ws.title for ws in sh.worksheets()]
    print("Worksheets:", names)
    ws = sh.worksheet(TAB_NAME)
except Exception as e:
    raise SystemExit(f"Cannot open spreadsheet/tab: {e}")

# Ensure DEBUG tab exists
try:
    debug_ws = sh.worksheet(DEBUG_TAB)
except:
    debug_ws = sh.add_worksheet(title=DEBUG_TAB, rows="200", cols="10")
    debug_ws.update('A1', [['timestamp','row','topic','videos','images','download_status','final_file','notes']])
print("Debug sheet ready:", DEBUG_TAB)

# -------- Helpers --------
def query_pixabay_counts(query):
    h = {"User-Agent":"Mozilla/5.0"}
    vurl = f"https://pixabay.com/api/videos/?key={PIXABAY_API_KEY}&q={requests.utils.requote_uri(query)}&per_page=5"
    iurl = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={requests.utils.requote_uri(query)}&image_type=photo&per_page=5"
    try:
        vresp = requests.get(vurl, headers=h, timeout=15).json()
        iresp = requests.get(iurl, headers=h, timeout=15).json()
    except Exception as e:
        return {"error": str(e)}
    v_hits = vresp.get("hits", [])
    i_hits = iresp.get("hits", [])
    sample_video = v_hits[0]["videos"]["medium"]["url"] if v_hits else None
    sample_image = i_hits[0].get("largeImageURL") if i_hits else None
    return {"videos": len(v_hits), "sample_video": sample_video, "images": len(i_hits), "sample_image": sample_image}

def download_stream(url, outpath, min_bytes=50_000):
    headers = {"User-Agent":"Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, stream=True, timeout=30)
    except Exception as e:
        return False, f"request_failed:{e}"
    if r.status_code != 200:
        return False, f"bad_status:{r.status_code}"
    total = 0
    try:
        with open(outpath, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    total += len(chunk)
    except Exception as e:
        return False, f"write_failed:{e}"
    if total < min_bytes:
        return False, f"file_too_small:{total}"
    return True, total

def make_text_image(text, outpath, w=1280, h=720, bgcolor=(0,0,0), fg=(255,255,255)):
    # use DejaVu font if available
    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
    try:
        font = ImageFont.truetype(font_path, 48)
    except:
        font = ImageFont.load_default()
    img = Image.new("RGB", (w,h), color=bgcolor)
    draw = ImageDraw.Draw(img)
    # wrap text
    wrapped = textwrap.fill(text, width=28)
    # center text
    text_w, text_h = draw.multiline_textsize(wrapped, font=font)
    x = (w - text_w) // 2
    y = (h - text_h) // 2
    draw.multiline_text((x,y), wrapped, font=font, fill=fg, align="center")
    img.save(outpath)
    return outpath

def create_final_from_video(video_path, narration_text, out_path):
    # create narration mp3
    tts_file = out_path.replace(".mp4", "_tts.mp3")
    gTTS(text=narration_text, lang="en").save(tts_file)
    try:
        clip = VideoFileClip(video_path).subclip(0, min(8, VideoFileClip(video_path).duration))
    except Exception as e:
        return False, f"video_load_error:{e}"
    audio = AudioFileClip(tts_file)
    final = clip.set_audio(audio)
    final.write_videofile(out_path, codec="libx264", audio_codec="aac")
    return True, out_path

def create_final_from_image(image_path, narration_text, out_path):
    tts_file = out_path.replace(".mp4", "_tts.mp3")
    gTTS(text=narration_text, lang="en").save(tts_file)
    imgclip = ImageClip(image_path).set_duration(8).resize((1280,720))
    audio = AudioFileClip(tts_file)
    final = imgclip.set_audio(audio)
    final.write_videofile(out_path, codec="libx264", audio_codec="aac")
    return True, out_path

# -------- Check Column A and add sample topics if empty --------
colA = ws.col_values(1)
if len(colA) <= 1:
    print("Column A is empty or only header — adding 3 sample topics.")
    samples = [
        "Ancient Secret of Vishnu in Mathura",
        "Legendary Tale of Durga in Kurukshetra",
        "Hidden Story of Shiva in Kashi"
    ]
    for s in samples:
        ws.append_row([s])
    time.sleep(1)  # let sheets propagate
else:
    print(f"Found {len(colA)-1} topics in Column A (skipping header if present).")

# re-read topics
topics = ws.col_values(1)[1:]  # skip header if present
print("First 20 topics (or fewer):", topics[:20])

# -------- Run checks for first up to 3 topics --------
debug_entries = []
for i, topic in enumerate(topics[:3], start=2):
    q = topic.strip()
    if not q:
        debug_ws.append_row([datetime.utcnow().isoformat(), i, topic, 0, 0, "empty_topic", "", ""], value_input_option="RAW")
        continue
    info = query_pixabay_counts(q)
    print(f"\nRow {i} query='{q}' -> videos={info.get('videos')} images={info.get('images')}")
    # attempt download video if available
    status = ""
    final_file = ""
    notes = ""
    if isinstance(info, dict) and info.get("videos",0) > 0 and info.get("sample_video"):
        sv = info["sample_video"]
        outv = os.path.join(LOCAL_DIR, f"debug_row{i}_video.mp4")
        ok, note = download_stream(sv, outv)
        status = f"video_dl_{ok}"
        notes = note
        if ok:
            # quick verify movie
            ok_read = True
            try:
                vclip = VideoFileClip(outv)
                dur = vclip.duration
                vclip.reader.close(); vclip.audio and vclip.audio.reader.close_proc()
            except Exception as e:
                ok_read = False
                notes = f"movie_read_err:{e}"
            if ok_read:
                final_file = os.path.join(DRIVE_FOLDER, f"final_test_row{i}.mp4")
                ok_final, notef = create_final_from_video(outv, q, os.path.join(LOCAL_DIR, f"final_test_row{i}.mp4"))
                if ok_final:
                    shutil.copy(os.path.join(LOCAL_DIR, f"final_test_row{i}.mp4"), final_file)
                    status = "final_created_from_video"
                else:
                    status = "final_failed"
                    notes = notef
    elif isinstance(info, dict) and info.get("images",0) > 0 and info.get("sample_image"):
        si = info["sample_image"]
        outimg = os.path.join(LOCAL_DIR, f"debug_row{i}_image.jpg")
        ok, note = download_stream(si, outimg, min_bytes=2000)
        status = f"image_dl_{ok}"
        notes = note
        if ok:
            # create final video from image (no ImageMagick)
            final_local = os.path.join(LOCAL_DIR, f"final_test_row{i}.mp4")
            ok_final, notef = create_final_from_image(outimg, q, final_local)
            if ok_final:
                final_file = os.path.join(DRIVE_FOLDER, os.path.basename(final_local))
                shutil.copy(final_local, final_file)
                status = "final_created_from_image"
            else:
                status = "final_failed_image"
                notes = notef
    else:
        # No media -> make a PIL text image and create fallback video
        print(f"No pixabay hits for '{q}'. Creating text-image fallback.")
        fallback_img = os.path.join(LOCAL_DIR, f"fallback_text_row{i}.jpg")
        make_text_image(q, fallback_img)
        final_local = os.path.join(LOCAL_DIR, f"final_test_row{i}.mp4")
        ok_final, notef = create_final_from_image(fallback_img, q, final_local)
        if ok_final:
            final_file = os.path.join(DRIVE_FOLDER, os.path.basename(final_local))
            shutil.copy(final_local, final_file)
            status = "final_created_from_textimage"
        else:
            status = "final_failed_textimage"
            notes = notef

    # write debug row to DEBUG_AUTOTEST
    debug_ws.append_row([datetime.utcnow().isoformat(), i, q, info.get("videos") if isinstance(info, dict) else "err", info.get("images") if isinstance(info, dict) else "err", status, final_file, notes], value_input_option="RAW")

# -------- Summary and listing files ----------
print("\n--- SUMMARY ---")
print("Local videos folder:", LOCAL_DIR)
print("Local files:", os.listdir(LOCAL_DIR)[:50])
print("Drive folder (first 50):", os.listdir(DRIVE_FOLDER)[:50])
print("Check the DEBUG_AUTOTEST tab in your spreadsheet for per-row logs.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Mounting Drive...
Mounted at /content/drive
